In [1]:
from instance_reader import read_instance
import random
from Solucion import Solucion
from funciones_auxiliares import seleccionar_segun_probabilidad
from Low_levels import *
from Hiperheuristica import HiperHeuristica

In [2]:
# comenzamos leyendo una instancia
n=0
Ll1= LL_agregacion_ordenes_random( id=n) # o
n+=1
Ll2 = LL_eliminacion_pasillos_malos(id= n)
n+=1
Ll3 = LL_factibilizar_demanda(id=n)
n+=1
Ll4 = LL_swap_orden_danina_por_util(id=n) #Mejoró en 7
n+=1
Ll5 = LL_agregacion_ordenes_faciles(id=n)
n+=1
Ll6 = LL_eliminacion_pasillos_malos(id=n)
n+=1
Ll7 = LL_agregar_monoitems_faciles(id=n)
n+=1
Ll8 = LL_swap_k_ordenes_daninas_por_faciles(id=n)
n+=1
Ll9 = LL_agregar_ordenes_concentradas_en_items_comunes(id=n)
n+=1
Ll10 = LL_agregacion_pasillo_orden_top(id=n)
n+=1
Ll11 = LL_dinkelbach_un_iter(id=n)
n+=1
Ll12 = LL_fuerza_bruta_un_largo(id=n)

low_levels = [Ll1,Ll2,Ll3,Ll4,Ll5, Ll6, Ll7, Ll8,Ll9, Ll10, Ll11, Ll12]
print(n)
len(low_levels)


11


12

In [ ]:
soluciones = {}
mejoras = {}
instancias_analisis = {}
for tipo in ["a", "b"]:
    print(f"instancias tipo {tipo}")
    if tipo == "a":
        total = 20
    else:
        total = 15
    for i in range(1,total+1):
        print(f"instancia {tipo}{i}")
        if len(str(i)) == 1:
            instance = read_instance(f"datasets/{tipo}/instance_000{str(i)}.txt")
        else:
            instance = read_instance(f"datasets/{tipo}/instance_00{str(i)}.txt")

        orders = random.sample(instance.orders, round(0.2 * len(instance.orders)))
        runners = random.sample(instance.runners, len(instance.runners))
        solucion = Solucion(selected_orders=orders, selected_runners=runners, instance=instance)

        hiperHeuristica = HiperHeuristica(instancia=instance, V=0.01, low_levels=low_levels, solucion=solucion, tiempo=590)
        hiperHeuristica.instancia_name = f"{tipo}{i}"
        hiperHeuristica.implementar()

        soluciones[i] = {
            "OB": hiperHeuristica.mejor_sol.objective_value,
            "Fac": hiperHeuristica.mejor_sol.is_factible,
            "it": hiperHeuristica.it
        }
        print(soluciones[i])

        mejoras[i] = {
            "Mejoras": hiperHeuristica.mejoras,
            "Tiempos": hiperHeuristica.tiempo_mejoras,
            "secuencia": hiperHeuristica.secuencias_mejoras
        }

        # Uso de cada low-level
        matriz = hiperHeuristica.Q
        resultado = np.sum(matriz, axis=1) - 2
        print("Tiempo de ejecución:", round(hiperHeuristica.tiempo_termino_real - hiperHeuristica.tiempo_inicio_real, 2))
        print()
    instancias_analisis[tipo] = mejoras

instancias tipo a
instancia a1
Conteo = 1, s_t = 6.60 False, s_c = 0.35 False, s* = 0.35 False, secuencia = [1, 7]
Conteo = 2, s_t = 0.53 True, s_c = 0.53 True, s* = 0.53 True, secuencia = [7]
Conteo = 3, s_t = 0.66 False, s_c = 0.53 True, s* = 0.53 True, secuencia = [8]
Pasillos actuales: 116 | Rango de elección: [1, 115] | usados: set() | instancia: a1
Set parameter Username
Set parameter LicenseID to value 2671583
Academic license - for non-commercial use only - expires 2026-05-27
Tiempo real: 0.01 segundos | pasillos intentados: 90
Nueva mejor solución 1.2142857142857142 con 90 pasillos
Conteo = 4, s_t = 1.21 True, s_c = 1.21 True, s* = 1.21 True, secuencia = [5, 5, 6, 6, 1, 0, 8, 1, 11]
Conteo = 5, s_t = 0.00 False, s_c = 1.21 True, s* = 1.21 True, secuencia = [0, 6, 6, 6, 9, 1, 2]
Conteo = 6, s_t = 1.88 False, s_c = 1.21 True, s* = 1.21 True, secuencia = [4, 3, 4]
Conteo = 7, s_t = 54.50 False, s_c = 1.21 True, s* = 1.21 True, secuencia = [4, 7, 8, 3, 1]
Conteo = 8, s_t = 15.00 T

In [ ]:
import numpy as np
import pandas as pd
from instance_reader import read_instance

# Guardar resultados
resultados = []

# Recorrer tipos de instancia A y B
for tipo in ["a", "b"]:
    print(f"Instancias tipo {tipo.upper()}")
    total = 20 if tipo == "a" else 15

    for i in range(1, total + 1):
        nombre = f"{tipo.upper()}{i}"
        if len(str(i)) == 1:
            ruta = f"datasets/{tipo}/instance_000{i}.txt"
        else:
            ruta = f"datasets/{tipo}/instance_00{i}.txt"

        try:
            instancia = read_instance(ruta)

            # Ítems distintos por orden y por runner
            ordenes_items = [len(order.items) for order in instancia.orders]
            corredores_items = [len(runner.stock) for runner in instancia.runners]

            resultados.append({
                "Instancia": nombre,
                "Media Ítems por Orden": round(np.mean(ordenes_items), 2),
                "Desviación Ítems por Orden": round(np.std(ordenes_items), 2),
                "Media Ítems por Runner": round(np.mean(corredores_items), 2),
                "Desviación Ítems por Runner": round(np.std(corredores_items), 2)
            })
        except Exception as e:
            print(f"Error al procesar {nombre}: {e}")

# Guardar resultados a Excel
df = pd.DataFrame(resultados)
df.to_excel("estadisticas_items_instancias.xlsx", index=False)
print("Archivo Excel generado: estadisticas_items_instancias.xlsx")

In [ ]:
import pandas as pd
# Transforma secuencias de índices a nombres de clases
for tipo in ["a", "b"]:
    mejoras_tipo = instancias_analisis[tipo]
    for key in mejoras_tipo:
        secuencias_nombres = []
        for secuencia in mejoras_tipo[key]['secuencia']:
            nombres = [low_levels[i].__class__.__name__ for i in secuencia]
            secuencias_nombres.append(nombres)
        mejoras_tipo[key]['secuencia_nombres'] = secuencias_nombres

    # Crear DataFrame por instancia
    writer = pd.ExcelWriter("mejoras.xlsx", engine="openpyxl")

    for key in mejoras_tipo:
        df = pd.DataFrame({
            "Tiempo": mejoras_tipo[key]["Tiempos"],
            "Mejora": mejoras_tipo[key]["Mejoras"],
            "Secuencia (IDs)": mejoras_tipo[key]["secuencia"],
            "Secuencia (Nombres)": mejoras_tipo[key]["secuencia_nombres"]
        })
        df.to_excel(writer, sheet_name=f"{tipo}{key}", index=False)

writer.close()
print("Excel guardado como mejoras.xlsx")

In [ ]:
instancias_analisis["b"] = mejoras

In [ ]:
import pandas as pd

# Ruta del archivo original
file_path = "Porcentaje_mejora_2.xlsx"
excel_file = pd.ExcelFile(file_path)

# Analizar todas las hojas tipo "A"
sheet_names_A = [s for s in excel_file.sheet_names if s.startswith("A")]
# También las hojas tipo "B"
sheet_names_B = [s for s in excel_file.sheet_names if s.startswith("B")]

# Recolectar datos por low-level
lowlevel_data = []

for sheet in sheet_names_A + sheet_names_B:
    df = excel_file.parse(sheet)
    for _, row in df.iterrows():
        if isinstance(row["Nombre"], str):
            lowlevel_data.append({
                "Instancia": sheet,
                "Nombre": row["Nombre"],
                "Promedio": row["Promedio"],
                "Maximo": row["Maximo"],
                "Uso": row["Uso"],
                "% Mejora": row["% Mejora"] if "% Mejora" in row else 0.0
            })

# Crear DataFrame general de low-levels
df_lowlevels = pd.DataFrame(lowlevel_data)

# Agrupación por tipo de low-level con mejoras
resumen_ll = df_lowlevels.groupby("Nombre").agg({
    "Uso": ["sum", "mean"],
    "Promedio": "mean",
    "Maximo": "max",
    "% Mejora": ["mean", "min", "max"]
}).reset_index()

# Renombrar columnas
resumen_ll.columns = [
    "Low-Level", "Total de Uso", "Uso Promedio",
    "Tiempo Promedio", "Tiempo Máximo",
    "Mejora Promedio", "Mejora Mínima", "Mejora Máxima"
]
resumen_ll.sort_values(by="Total de Uso", ascending=False, inplace=True)

# Función para extraer métricas por grupo de hojas
def extraer_metricas(sheet_names):
    instancias = []
    tiempos_total_ll = []
    tiempos_total_algoritmo = []
    gaps = []
    iteraciones = []
    objetivos = []
    optimos = []

    for sheet in sheet_names:
        df = excel_file.parse(sheet)
        tail = df.tail(20)

        duracion_ll = None
        duracion_algoritmo = None
        gap = None
        iters = None
        obj = None
        opt = None

        for _, row in tail.iterrows():
            if isinstance(row[0], str):
                if "Duración total de low-levels" in row[0]:
                    duracion_ll = float(row["Nombre"]) if pd.notnull(row["Nombre"]) else None
                elif "Duración total del algoritmo" in row[0]:
                    duracion_algoritmo = float(row["Nombre"]) if pd.notnull(row["Nombre"]) else None
                elif "gap" in row[0].lower() or "GAP" in row[0]:
                    gap = float(row["Nombre"]) if pd.notnull(row["Nombre"]) else None
                elif "Iteraciones" in row[0]:
                    iters = float(row["Nombre"]) if pd.notnull(row["Nombre"]) else None
                elif "Objective value" in row[0]:
                    obj = float(row["Nombre"]) if pd.notnull(row["Nombre"]) else None
                elif "Valor óptimo" in row[0]:
                    opt = float(row["Nombre"]) if pd.notnull(row["Nombre"]) else None

        if duracion_ll is not None:
            instancias.append(sheet)
            tiempos_total_ll.append(duracion_ll)
            tiempos_total_algoritmo.append(duracion_algoritmo)
            gaps.append(gap)
            iteraciones.append(iters)
            objetivos.append(obj)
            optimos.append(opt)

    df_metricas = pd.DataFrame({
        "Instancia": instancias,
        "Duración total LL (s)": tiempos_total_ll,
        "Duración total algoritmo (s)": tiempos_total_algoritmo,
        "Gap (%)": gaps,
        "Iteraciones": iteraciones,
        "Objective value": objetivos,
        "Valor óptimo": optimos
    })
    return df_metricas

# Métricas por instancia A y B
df_metricas_A = extraer_metricas(sheet_names_A)
df_metricas_B = extraer_metricas(sheet_names_B)

# Guardar todos los resultados en un nuevo Excel
with pd.ExcelWriter("lowlevels_analisis_mejora_2.xlsx", engine="openpyxl") as writer:
    resumen_ll.to_excel(writer, sheet_name="Resumen_LowLevels", index=False)
    df_metricas_A.to_excel(writer, sheet_name="Resumen_Instancias_A", index=False)
    df_metricas_B.to_excel(writer, sheet_name="Resumen_Instancias_B", index=False)

# Mostrar estadísticas adicionales
iter_max = df_metricas_A["Iteraciones"].max()
iter_mean = df_metricas_A["Iteraciones"].mean()
print(f"Iteraciones máximas (A): {iter_max}")
print(f"Iteraciones promedio (A): {iter_mean:.2f}")